# Regression Model with Tensorflow Decision Forests (remote)

In [1]:
!pip install -q --user --no-warn-conflicts google-cloud-aiplatform pandas-gbq
from datetime import datetime

import pandas as pd

from google.cloud import aiplatform

aiplatform.init(location='europe-west4')

## Load Data with BigQuery

In [2]:
df_csv = pd.read_csv('gs://sandbox-michael-menzel-data-europe-west4/advertising-regression-data/advertising_with_header.csv')
df_csv.to_gbq('example_dataset.advertising_regression', if_exists='replace')

  0%|          | 0/1 [00:00<?, ?it/s]

200 out of 200 rows loaded.


100%|██████████| 1/1 [00:00<00:00, 1278.75it/s]


## Training Script

In [3]:
%%writefile build/tfdf_reg_train.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
from datetime import datetime
import io
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_decision_forests as tfdf
import tensorflow_addons as tfa

from google.cloud import bigquery

import hypertune

hpt = hypertune.HyperTune()

file_writer = tf.summary.create_file_writer(os.environ['AIP_TENSORBOARD_LOG_DIR'])

class MetricReporter(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            for metric in logs:
                print(f"Reporting {metric}: {logs[metric]} in epoch: {epoch}")
                hpt.report_hyperparameter_tuning_metric(
                    hyperparameter_metric_tag=metric,
                    metric_value=logs[metric],
                    global_step=epoch)

def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    return tf.expand_dims(tf.image.decode_png(buf.getvalue(), channels=4), 0)

def log_prediction_scatter(epoch, logs):
    for title, dataset in DATASETS.items():
        test_pred = tf.cast(tf.squeeze(model.predict(dataset.unbatch().batch(1))), tf.float64)
        test_truth = tf.squeeze(LABELS[title])
        
        df = pd.DataFrame({'predicted': test_pred, 'truth': test_truth})
        df['error'] = tf.subtract(test_pred, test_truth)
        
        figure, ax = plt.subplots()
        sns.scatterplot(data=df.sort_values('truth').reset_index(drop=True), ax=ax)
        plt.title(f'Prediction & Error Scatter ({title}):')
        scatter_image = plot_to_image(figure)

        with file_writer.as_default():
            tf.summary.image(f'Prediction vs. Truth ({title})', scatter_image, step=epoch)




query_job = bigquery.Client(project='sandbox-michael-menzel').query('select * from example_dataset.advertising_regression')
df = query_job.result().to_dataframe()
TARGET = 'Sales'

test_split = int(len(df[TARGET]) * 0.1)
print(f'Splitting dataset into {len(df[TARGET]) - test_split} items for training and {test_split} items for test.')

dataset_train = (tfdf.keras.pd_dataframe_to_tf_dataset(df[test_split:], label=TARGET, task=tfdf.keras.Task.REGRESSION)
                 .cache())
dataset_test = (tfdf.keras.pd_dataframe_to_tf_dataset(df[:test_split], label=TARGET, task=tfdf.keras.Task.REGRESSION)
                .cache())

train_labels = list(dataset_train.map(lambda x, y: y))
test_labels = list(dataset_test.map(lambda x, y: y))

LABELS = {'Test': test_labels, 'Train': train_labels}
DATASETS = {'Test': dataset_test, 'Train': dataset_train}


def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))

model = tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.REGRESSION)
model.compile(metrics=[rmse, 'mse', 'mae', 'mape', tfa.metrics.RSquare(y_shape=(1,))])
history = model.fit(dataset_train,
                    callbacks=[
                        MetricReporter(),
                        tf.keras.callbacks.LambdaCallback(on_epoch_end=log_prediction_scatter),
                        tf.keras.callbacks.TensorBoard(
                            log_dir=os.getenv("AIP_TENSORBOARD_LOG_DIR"))
                    ])

print('Evaluating the model...')
model.evaluate(dataset_test, callbacks=[tf.keras.callbacks.TensorBoard(log_dir=os.getenv("AIP_TENSORBOARD_LOG_DIR"))])

model.save(os.getenv("AIP_MODEL_DIR"))


Overwriting build/tfdf_reg_train.py


## Start Training Script on Vertex AI

In [4]:
TIMESTAMP = int(datetime.now().timestamp())
JOB_NAME = f'tfdf-sales-regression-training-{TIMESTAMP}'
TENSORBOARD = 'projects/sandbox-michael-menzel/locations/europe-west4/tensorboards/2056808023568941056'
SERVICE_ACCOUNT = !gcloud config get-value account
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

vertex_ai_job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path='build/tfdf_reg_train.py',
    container_uri='europe-docker.pkg.dev/vertex-ai/training/tf-cpu.2-6:latest',
    staging_bucket='gs://sandbox-michael-menzel-training-europe-west4/tfdf-sales-regression-training/',
    requirements=['tensorflow_decision_forests', 'tensorflow-addons'],
    model_serving_container_image_uri='europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest',
)

vertex_ai_model = vertex_ai_job.run(
    replica_count=1, 
    machine_type='n1-standard-4',
    tensorboard=TENSORBOARD,
    service_account=SERVICE_ACCOUNT,
    args=[]
)

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://sandbox-michael-menzel-training-europe-west4/tfdf-sales-regression-training/aiplatform-2022-01-23-15:26:26.419-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://sandbox-michael-menzel-training-europe-west4/tfdf-sales-regression-training/aiplatform-custom-training-2022-01-23-15:26:26.510 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/8404581120812777472?project=928871478446
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/928871478446/locations/europe-west4/trainingPipelines/8404581120812777472 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/8475231339967152128?project=928871478446
INF